In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import matplotlib.pyplot as plt
import io
import base64
import pickle
from flask_cors import CORS

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes
# Load the dataset
df_list = pd.read_html('/home/fares/DatasetsOrange/offer.xls')
df = df_list[0]
df['Period'] = pd.to_datetime(df['Period'])  # Convert the index column to Timestamp objects
df = df.set_index('Period')
# Preprocess the dataset
float_cols = df.columns
df[float_cols] = df[float_cols].applymap(lambda x: str(x).replace(",", "."))
df[float_cols] = df[float_cols].applymap(lambda x: str(x).replace(" ", ""))
df[float_cols] = df[float_cols].apply(pd.to_numeric, errors='coerce', axis=1)
df[float_cols] = df[float_cols].applymap(lambda x: x / (1024 * 1024 * 1024))

# Model file path
model_path = 'ARIMA.pkl'

# Load the ARIMA model
with open(model_path, 'rb') as file:
    model_fit = pickle.load(file)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()  # Get JSON data from the request

    column_name = data['column_name']
    desired_days = data['desired_days']

    # Generate a range of dates for prediction
    start_date = df.index[-1] + pd.DateOffset(days=1)
    end_date = start_date + pd.DateOffset(days=desired_days)

    # Convert start_date and end_date to UTC timestamps
    start_date_utc = start_date.tz_localize(None)
    end_date_utc = end_date.tz_localize(None)

    # Generate predictions using the loaded ARIMA model
    predictions = model_fit.predict(start=start_date_utc, end=end_date_utc)

    # Generate prediction dates in the local timezone
    prediction_dates_local = pd.date_range(start=start_date, end=end_date, tz=df.index.tz)

    # Create a list of dictionaries with dates and predictions
    predictions_with_dates = [
        {'date': date.strftime('%Y-%m-%d %H:%M:%S'), 'prediction': prediction}
        for date, prediction in zip(prediction_dates_local, predictions)
    ]

    # Create response JSON
    response = {
        'predictions': predictions_with_dates
    }

    return jsonify(response)
@app.route('/descriptive-curve', methods=['POST'])
def descriptive_curve():
    try:
        data = request.get_json()

        # Get input parameters from the request
        column_name = data['column_name']  # Corrected this line
        start_date = data['start_date']  # Add this line to get the start date
        end_date = data['end_date']  # Add this line to get the end date

        # Subset the dataset based on the specified date range
        subset_df = df.loc[start_date:end_date, column_name]

        # Get the values and dates of the descriptive curve
        values = subset_df.values.tolist()
        dates = subset_df.index.strftime('%Y-%m-%d').tolist()
        data = [{'date': date, 'values': value} for date, value in zip(dates, values)]

        # Create the response
        response = {
            'Data': data,
        }

        return jsonify(response)

    except Exception as e:
        error_message = str(e)
        response = {
            'error': error_message
        }
        return jsonify(response), 500
@app.route('/all', methods=['POST'])
def dataset_values():
    try:
        data = request.get_json()  # Get JSON data from the request

        column_name = data['column_name']

        # Get the values and dates for the specified column
        values = df[column_name].values.tolist()
        dates = df.index.strftime('%Y-%m-%d').tolist()

        # Create a list of dictionaries with dates and values
        data = [{'date': date, 'values': value} for date, value in zip(dates, values)]

        # Create the response
        response = {
            'Data': data
        }

        return jsonify(response)

    except Exception as e:
        error_message = str(e)
        response = {
            'error': error_message
        }
        return jsonify(response), 500


if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Jul/2023 08:56:25] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:56:31] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:56:32] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2023 08:56:34] "POST /predict HTTP/1.1" 200 -
